In [73]:
# 1. Montar o Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 2. Importar bibliotecas
import pandas as pd
import numpy as np

# 3. Definir caminho do arquivo no Drive
caminho_prospect = "/content/drive/My Drive/Tech Challenge 5/Dados/silver/dados_processed/prospects_processed.parquet"
caminho_vagas = "/content/drive/My Drive/Tech Challenge 5/Dados/silver/dados_processed/vagas_processed.parquet"
caminho_application = "/content/drive/My Drive/Tech Challenge 5/Dados/silver/dados_processed/application_processed.parquet"

# 4. Ler o parquet direto no pandas
df_prospect = pd.read_parquet(caminho_prospect)
df_vagas = pd.read_parquet(caminho_vagas)
df_application = pd.read_parquet(caminho_application)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# PROSPECT

In [74]:
df_prospect.head()

,job_id,titulo,prospect_codigo,prospect_situacao_candidado,prospect_data_candidatura,prospect_data_ultima_atualizacao,prospect_situacao_candidado_normalized,titulo_nivel_senioridade,titulo_cleaned
0,2,Analista de Negocios SR,12585,Contratado pela Decision,2018-12-04,2018-12-04,Contratado,Senior,Analista de Negocios SR
1,3,Arquiteto de Sistemas SR,12598,Encaminhado ao Requisitante,2018-12-06,2019-01-15,encaminhado ao requisitante,Senior,Arquiteto de Sistemas SR
2,3,Arquiteto de Sistemas SR,12595,Não Aprovado pelo Cliente,2018-12-05,2018-12-07,Aprovado,Senior,Arquiteto de Sistemas SR
3,4,Analista de Projetos SR,12618,Encaminhado ao Requisitante,2018-12-07,2019-01-14,encaminhado ao requisitante,Senior,Analista de Projetos SR
4,5,Analista de Sistemas SR,12626,Encaminhado ao Requisitante,2018-12-10,2019-01-15,encaminhado ao requisitante,Senior,Analista de Sistemas SR


In [75]:
df_prospect.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53759 entries, 0 to 53758
Data columns (total 9 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   job_id                                  53759 non-null  int64 
 1   titulo                                  53759 non-null  object
 2   prospect_codigo                         53759 non-null  object
 3   prospect_situacao_candidado             53759 non-null  object
 4   prospect_data_candidatura               53759 non-null  object
 5   prospect_data_ultima_atualizacao        49846 non-null  object
 6   prospect_situacao_candidado_normalized  53759 non-null  object
 7   titulo_nivel_senioridade                53759 non-null  object
 8   titulo_cleaned                          53759 non-null  object
dtypes: int64(1), object(8)
memory usage: 3.7+ MB


In [76]:
df_prospect_filtrado = df_prospect[["job_id","prospect_codigo","titulo"]]
df_prospect_filtrado.head()

,job_id,prospect_codigo,titulo
0,2,12585,Analista de Negocios SR
1,3,12598,Arquiteto de Sistemas SR
2,3,12595,Arquiteto de Sistemas SR
3,4,12618,Analista de Projetos SR
4,5,12626,Analista de Sistemas SR


In [77]:
contagem_distintos_df = (
    df_prospect_filtrado.nunique()
    .reset_index()
    .rename(columns={"index": "coluna", 0: "qtd_distintos"})
)

print(contagem_distintos_df)

            coluna  qtd_distintos
0           job_id          11279
1  prospect_codigo          29405
2           titulo           9910


In [78]:
# conta valores nulos
qtd_nulos = df_prospect_filtrado.isna().sum()

# conta strings vazias ("") ou só com espaços
qtd_vazios = (df_prospect_filtrado.applymap(
    lambda x: isinstance(x, str) and x.strip() == "")
).sum()

# junta em um único DataFrame
resumo_vazios = pd.DataFrame({
    "coluna": df_prospect_filtrado.columns,
    "qtd_nulos": qtd_nulos.values,
    "qtd_vazios": qtd_vazios.values,
    "total_vazios": (qtd_nulos + qtd_vazios).values,
    "pct_vazios": ((qtd_nulos + qtd_vazios) / len(df_prospect_filtrado) * 100).round(2).values
})

print(resumo_vazios)


            coluna  qtd_nulos  qtd_vazios  total_vazios  pct_vazios
0           job_id          0           0             0         0.0
1  prospect_codigo          0           0             0         0.0
2           titulo          0           0             0         0.0


/tmp/ipython-input-1405101537.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  qtd_vazios = (df_prospect_filtrado.applymap(


# VAGAS

In [79]:
df_vagas.head()

,id,data_requicisao,limite_esperado_para_contratacao,titulo_vaga,vaga_sap,cliente,tipo_contratacao,prazo_contratacao,objetivo_vaga,prioridade_vaga,...,valor_compra_1,valor_compra_2,data_inicial,data_final,habilidades_comportamentais_necessarias,areas_atuacao_cleaned,principais_atividades_cleaned,competencia_tecnicas_e_comportamentais_cleaned,demais_observacoes_cleaned,habilidades_comportamentais_necessarias_cleaned
0,5185,2021-05-04,0000-00-00,Operation Lead -,Não,"Morris, Moran and Dodson",CLT Full,None,None,Média,...,R$,None,None,None,None,TI Sistemas e Ferramentas,Operations Lead Roles & Responsibilities: • Th...,Required Skills: • Prior experience in Cloud I...,100% Remoto Período – entre 5 – 6 meses,None
1,5184,2021-05-04,0000-00-00,Consultor PP/QM Sênior,Não,"Morris, Moran and Dodson",CLT Full,None,Contratação,Média,...,R$,None,None,None,None,TI Desenvolvimento/Programação,Consultor PP/QM Sr. • Consultor PP/QM Sênior c...,• Consultor PP/QM Sênior com experiencia em pr...,• Início: Imediato • Fim: Jan/22,None
2,5183,2021-05-04,0000-00-00,ANALISTA PL/JR C/ SQL,Não,"Morris, Moran and Dodson",CLT Full,None,RFP,Média,...,R$,None,None,None,None,TI Sistemas e Ferramentas,Descrição – Atividades: o Monitoramento das in...,Requisitos mandatórios: o Conhecimentos Técnic...,Localização: Remoto Perfil: Analista Pleno ou ...,None
3,5182,2021-05-04,2021-05-18,Technical Architect - 11894809,Não,Nelson-Page,"PJ/Autônomo, CLT Full",Determinado,Contratação,Alta: Alta complexidade 3 a 5 dias,...,fechado,None,2021-05-18,2022-01-17,None,TI Projetos,Descrição/Comentário: Architecture Frameworks ...,Descrição/Comentário: Architecture Frameworks ...,Budgeted Rate - indicate currency and type (ho...,None
4,5181,2021-05-04,0000-00-00,Consultor SAP AUTHORIZATION (BCA) -Pleno / Sênior,Não,Mann and Sons,CLT Full,None,None,Média,...,R$,None,None,None,None,TI SAP,Experiência como Consultor SAP AUTHORIZATION (...,Experiência como Consultor SAP AUTHORIZATION (...,contratação CLT full pela Decision locação rem...,None


In [80]:
df_vagas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14081 entries, 0 to 14080
Data columns (total 36 columns):
 #   Column                                           Non-Null Count  Dtype 
---  ------                                           --------------  ----- 
 0   id                                               14081 non-null  int64 
 1   data_requicisao                                  14081 non-null  object
 2   limite_esperado_para_contratacao                 14081 non-null  object
 3   titulo_vaga                                      14081 non-null  object
 4   vaga_sap                                         14081 non-null  object
 5   cliente                                          14081 non-null  object
 6   tipo_contratacao                                 13477 non-null  object
 7   prazo_contratacao                                9779 non-null   object
 8   objetivo_vaga                                    10994 non-null  object
 9   prioridade_vaga                        

In [81]:
df_vagas_filtrado = df_vagas[["id","areas_atuacao", "titulo_vaga","objetivo_vaga","principais_atividades","competencia_tecnicas_e_comportamentais","estado","nivel_academico","nivel_ingles","nivel_espanhol","nivel_profissional"]]
df_vagas_filtrado.head()

,id,areas_atuacao,titulo_vaga,objetivo_vaga,principais_atividades,competencia_tecnicas_e_comportamentais,estado,nivel_academico,nivel_ingles,nivel_espanhol,nivel_profissional
0,5185,TI - Sistemas e Ferramentas-,Operation Lead -,None,Operations Lead\n\nRoles & Responsibilities:\n...,Required Skills:\n• Prior experience in Cloud ...,São Paulo,Ensino Superior Completo,Avançado,Fluente,Sênior
1,5184,TI - Desenvolvimento/Programação-,Consultor PP/QM Sênior,Contratação,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...,• Consultor PP/QM Sênior com experiencia em pr...,São Paulo,Ensino Superior Completo,Fluente,Nenhum,Sênior
2,5183,TI - Sistemas e Ferramentas-,ANALISTA PL/JR C/ SQL,RFP,Descrição – Atividades:\n\no Monitoramento das...,Requisitos mandatórios:\n\no Conhecimentos Téc...,São Paulo,Ensino Superior Completo,Nenhum,Intermediário,Analista
3,5182,TI - Projetos-,Technical Architect - 11894809,Contratação,Descrição/Comentário: Architecture Frameworks ...,Descrição/Comentário: Architecture Frameworks ...,São Paulo,Ensino Superior Completo,Básico,Básico,Analista
4,5181,TI - SAP-,Consultor SAP AUTHORIZATION (BCA) -Pleno / Sênior,None,Experiência como Consultor SAP AUTHORIZATION (...,Experiência como Consultor SAP AUTHORIZATION (...,São Paulo,Ensino Superior Completo,Intermediário,Nenhum,Sênior


In [82]:
contagem_distintos_vagas = (
    df_vagas_filtrado.nunique()
    .reset_index()
    .rename(columns={"index": "coluna", 0: "qtd_distintos"})
)

print(contagem_distintos_vagas)

                                    coluna  qtd_distintos
0                                       id          14081
1                            areas_atuacao             73
2                              titulo_vaga          12452
3                            objetivo_vaga              5
4                    principais_atividades          11708
5   competencia_tecnicas_e_comportamentais          11550
6                                   estado             26
7                          nivel_academico             16
8                             nivel_ingles              6
9                           nivel_espanhol              6
10                      nivel_profissional             14


In [83]:
# contar nulos
qtd_nulos = df_vagas_filtrado.isna().sum()

# contar strings vazias ou só com espaços
qtd_vazios = (df_vagas_filtrado.applymap(
    lambda x: isinstance(x, str) and x.strip() == "")
).sum()

# consolidar em um DataFrame
resumo_vazios_vagas = pd.DataFrame({
    "coluna": df_vagas_filtrado.columns,
    "qtd_nulos": qtd_nulos.values,
    "qtd_vazios": qtd_vazios.values,
    "total_vazios": (qtd_nulos + qtd_vazios).values,
    "pct_vazios": ((qtd_nulos + qtd_vazios) / len(df_vagas_filtrado) * 100).round(2).values
})

print(resumo_vazios_vagas)


                                    coluna  qtd_nulos  qtd_vazios  \
0                                       id          0           0   
1                            areas_atuacao         15           0   
2                              titulo_vaga          0           0   
3                            objetivo_vaga       3087           0   
4                    principais_atividades          4           0   
5   competencia_tecnicas_e_comportamentais          7           0   
6                                   estado         11           0   
7                          nivel_academico          0           0   
8                             nivel_ingles          0           0   
9                           nivel_espanhol       1304           0   
10                      nivel_profissional          0           0   

    total_vazios  pct_vazios  
0              0        0.00  
1             15        0.11  
2              0        0.00  
3           3087       21.92  
4              4

/tmp/ipython-input-2065673334.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  qtd_vazios = (df_vagas_filtrado.applymap(


# APPLICATION

In [84]:
df_application.head()

,job_id,objetivo_profissional,data_criacao,local,sabendo_de_nos_por,data_atualizacao,codigo_profissional,data_aceite,fonte_indicacao,telefone_celular,...,cursos,ano_conclusao,data_admissao,data_ultima_promocao,conhecimentos_tecnicos_list,certificacoes_list,outras_certificacoes_list,remuneracao_numeric,cv_pt_cleaned,telefone_celular_normalized
0,31000,,2021-11-10,,,2021-11-10,31000,None,None,(11) 97048-2708,...,None,None,None,None,[],[],[],NaN,assistente administrativo santosbatista itapec...,11970482708
1,31001,Analista Administrativo,2021-11-10,"São Paulo, São Paulo",Outros,2021-11-11,31001,None,None,(11) 93723-4396,...,None,None,None,None,[],[],[],1900.0,formação acadêmica ensino médio (2º grau) em e...,11937234396
2,31002,Administrativo | Financeiro,2021-11-10,"São Paulo, São Paulo",Anúncio,2021-11-10,31002,None,None,(11) 92399-9824,...,Administração de Empresas,2012,None,None,[],"[MS [77-418] MOS: Microsoft Office Word 2013, ...",[],NaN,objetivo: área administrativa | financeira res...,11923999824
3,31003,Área administrativa,2021-11-10,"São Paulo, São Paulo",Site de Empregos,2021-11-10,31003,None,None,(11) 98100-1727,...,None,None,None,None,[],[],[],1100.0,formação ensino médio completo informática int...,11981001727
4,31004,,2021-11-10,,,2021-11-10,31004,None,None,(11) 92517-2678,...,None,None,None,None,[],[],[],NaN,última atualização em 09/11/2021 ­ sp ensino s...,11925172678


In [85]:
df_application.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42482 entries, 0 to 42481
Data columns (total 37 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   job_id                       42482 non-null  object 
 1   objetivo_profissional        42482 non-null  object 
 2   data_criacao                 42480 non-null  object 
 3   local                        42482 non-null  object 
 4   sabendo_de_nos_por           42482 non-null  object 
 5   data_atualizacao             42480 non-null  object 
 6   codigo_profissional          42482 non-null  object 
 7   data_aceite                  11329 non-null  object 
 8   fonte_indicacao              0 non-null      object 
 9   telefone_celular             42482 non-null  object 
 10  sexo                         7184 non-null   object 
 11  estado_civil                 7451 non-null   object 
 12  pcd                          6321 non-null   object 
 13  endereco        

In [86]:
df_application_filtrado = df_application[["job_id","sexo","pcd","cv_pt_cleaned"]]
df_application_filtrado.head()

,job_id,sexo,pcd,cv_pt_cleaned
0,31000,None,None,assistente administrativo santosbatista itapec...
1,31001,Feminino,Não,formação acadêmica ensino médio (2º grau) em e...
2,31002,Feminino,Não,objetivo: área administrativa | financeira res...
3,31003,Feminino,Não,formação ensino médio completo informática int...
4,31004,None,None,última atualização em 09/11/2021 ­ sp ensino s...


In [87]:
contagem_distintos_application = (
    df_application_filtrado.nunique()
    .reset_index()
    .rename(columns={"index": "coluna", 0: "qtd_distintos"})
)

print(contagem_distintos_application)

          coluna  qtd_distintos
0         job_id          42482
1           sexo              2
2            pcd              2
3  cv_pt_cleaned          28721


In [88]:
# contar nulos
qtd_nulos = df_application_filtrado.isna().sum()

# contar strings vazias ou só com espaços
qtd_vazios = (df_application_filtrado.applymap(
    lambda x: isinstance(x, str) and x.strip() == "")
).sum()

# consolidar em um DataFrame
resumo_vazios_app = pd.DataFrame({
    "coluna": df_application_filtrado.columns,
    "qtd_nulos": qtd_nulos.values,
    "qtd_vazios": qtd_vazios.values,
    "total_vazios": (qtd_nulos + qtd_vazios).values,
    "pct_vazios": ((qtd_nulos + qtd_vazios) / len(df_application_filtrado) * 100).round(2).values
})

print(resumo_vazios_app)


          coluna  qtd_nulos  qtd_vazios  total_vazios  pct_vazios
0         job_id          0           0             0        0.00
1           sexo      35298           0         35298       83.09
2            pcd      36161           0         36161       85.12
3  cv_pt_cleaned      13683           0         13683       32.21


/tmp/ipython-input-3853170318.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  qtd_vazios = (df_application_filtrado.applymap(


# JOIN

In [89]:
# Garantir que as chaves tenham o mesmo tipo
df_vagas_filtrado["id"] = df_vagas_filtrado["id"].astype(str)
df_application_filtrado["job_id"] = df_application_filtrado["job_id"].astype(str)
df_prospect_filtrado["job_id"] = df_prospect_filtrado["job_id"].astype(str)

# LEFT JOIN df_vagas -> df_application  ON df_application.job_id = df_vagas.id
df_join = df_vagas_filtrado.merge(
    df_application_filtrado,
    how="left",
    left_on="id",
    right_on="job_id",
    suffixes=("_vaga", "_app")  # se houver nomes de colunas iguais
)

# LEFT JOIN (resultado) -> df_prospect ON df_prospect.job_id = df_vagas.id
df_join = df_join.merge(
    df_prospect_filtrado,
    how="left",
    left_on="id",
    right_on="job_id",
    suffixes=("", "_prospect")
)

# (Opcional) manter só a chave principal 'id' e remover chaves duplicadas vindas dos rights
for col in ["job_id", "job_id_prospect"]:
    if col in df_join.columns:
        df_join = df_join.drop(columns=col)

print(df_join.shape)
df_join.head()


/tmp/ipython-input-1434402877.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vagas_filtrado["id"] = df_vagas_filtrado["id"].astype(str)
/tmp/ipython-input-1434402877.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_application_filtrado["job_id"] = df_application_filtrado["job_id"].astype(str)
/tmp/ipython-input-1434402877.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

(56539, 16)


,id,areas_atuacao,titulo_vaga,objetivo_vaga,principais_atividades,competencia_tecnicas_e_comportamentais,estado,nivel_academico,nivel_ingles,nivel_espanhol,nivel_profissional,sexo,pcd,cv_pt_cleaned,prospect_codigo,titulo
0,5185,TI - Sistemas e Ferramentas-,Operation Lead -,None,Operations Lead\n\nRoles & Responsibilities:\n...,Required Skills:\n• Prior experience in Cloud ...,São Paulo,Ensino Superior Completo,Avançado,Fluente,Sênior,None,None,None,11010,Operation Lead -
1,5184,TI - Desenvolvimento/Programação-,Consultor PP/QM Sênior,Contratação,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...,• Consultor PP/QM Sênior com experiencia em pr...,São Paulo,Ensino Superior Completo,Fluente,Nenhum,Sênior,None,None,None,26770,Consultor PP/QM Sênior
2,5184,TI - Desenvolvimento/Programação-,Consultor PP/QM Sênior,Contratação,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...,• Consultor PP/QM Sênior com experiencia em pr...,São Paulo,Ensino Superior Completo,Fluente,Nenhum,Sênior,None,None,None,26759,Consultor PP/QM Sênior
3,5184,TI - Desenvolvimento/Programação-,Consultor PP/QM Sênior,Contratação,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...,• Consultor PP/QM Sênior com experiencia em pr...,São Paulo,Ensino Superior Completo,Fluente,Nenhum,Sênior,None,None,None,26758,Consultor PP/QM Sênior
4,5184,TI - Desenvolvimento/Programação-,Consultor PP/QM Sênior,Contratação,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...,• Consultor PP/QM Sênior com experiencia em pr...,São Paulo,Ensino Superior Completo,Fluente,Nenhum,Sênior,None,None,None,26757,Consultor PP/QM Sênior


In [91]:
df_join["nivel_profissional"].unique()

array(['Sênior', 'Analista', 'Pleno', 'Coordenador', 'Especialista',
       'Gerente', 'Supervisor', 'Assistente', 'Líder', 'Júnior',
       'Aprendiz', 'Auxiliar', 'Trainee', 'Técnico de Nível Médio'],
      dtype=object)

In [92]:
#     Taxonomia de senioridade

SENIORIDADE = {
    "Aprendiz": "Estagiário",
    "Trainee": "Estagiário",
    "Estagiário": "Estagiário",
    "Assistente": "Assistente",
    "Auxiliar": "Assistente",
    "Técnico de Nível Médio": "Assistente",
    "Júnior": "Analista Júnior",
    "Pleno": "Analista Pleno",
    "Sênior": "Analista Sênior",
    "Analista": "Analista Pleno", # default quando vier "Analista" sem sufixo
    "Especialista": "Especialista",
    "Líder": "Líder",
    "Supervisor": "Supervisor",
    "Coordenador": "Coordenador",
    "Gerente": "Gerente",
    "Head": "Head",
    "Diretor": "Diretor",
    "CEO": "CEO",
}

# Código Final

In [ ]:
# 1. Conectar o Google Colab ao Google Drive para acessar os arquivos diretamente de lá.
from google.colab import drive
drive.mount('/content/drive')

# 2. Importar bibliotecas
import pandas as pd
import numpy as np

# 3. Definir caminho do arquivo no Drive
caminho_prospect = "/content/drive/My Drive/Tech Challenge 5/Dados/silver/dados_processed/prospects_processed.parquet"
caminho_vagas = "/content/drive/My Drive/Tech Challenge 5/Dados/silver/dados_processed/vagas_processed.parquet"
caminho_application = "/content/drive/My Drive/Tech Challenge 5/Dados/silver/dados_processed/application_processed.parquet"

# 4. Ler o parquet direto no pandas
df_prospect = pd.read_parquet(caminho_prospect)
df_vagas = pd.read_parquet(caminho_vagas)
df_application = pd.read_parquet(caminho_application)

# 4.1 Selecionar colunas relevantes (criar *_filtrado)
df_vagas_filtrado = df_vagas[[
    "id",
    "titulo_vaga",
    "objetivo_vaga",
    "principais_atividades",
    "competencia_tecnicas_e_comportamentais",
    "estado",
    "nivel_academico",
    "nivel_ingles",
    "nivel_espanhol",
    "nivel_profissional"
]]

df_application_filtrado = df_application[[
    "job_id",
    "sexo",
    "pcd",
    "cv_pt"
]]

df_prospect_filtrado = df_prospect[[
    "job_id",
    "prospect_codigo",
    "titulo",
    "prospect_situacao_candidado_normalized"
]]

# -----------------------------
# Dicionário de Senioridade
# -----------------------------
# Estes dicionários definem a taxonomia de níveis profissionais.
# SENIORIDADE_GROUP converte as categorias originais em grupos padronizados
# SENIORIDADE_LVL atribui um número (de 1 a 13) a cada grupo.
SENIORIDADE_GROUP = {
    "Aprendiz": "Estagiário",
    "Trainee": "Estagiário",
    "Estagiário": "Estagiário",
    "Assistente": "Assistente",
    "Auxiliar": "Assistente",
    "Técnico de Nível Médio": "Assistente",
    "Júnior": "Analista Júnior",
    "Pleno": "Analista Pleno",
    "Analista": "Analista Pleno",      # usado como padrão quando vier só "Analista"
    "Sênior": "Analista Sênior",
    "Especialista": "Especialista",
    "Líder": "Líder",
    "Supervisor": "Supervisor",
    "Coordenador": "Coordenador",
    "Gerente": "Gerente",
    "Head": "Head",
    "Diretor": "Diretor",
    "CEO": "CEO"
}

DE_PARA = {
    "senioridade_lvl" : {
        "Estagiário": 1,
        "Assistente": 2,
        "Analista Júnior": 3,
        "Analista Pleno": 4,
        "Analista Sênior": 5,
        "Especialista": 6,
        "Líder": 7,
        "Supervisor": 8,
        "Coordenador": 9,
        "Gerente": 10,
        "Head": 11,
        "Diretor": 12,
        "CEO": 13
    }
}

DE_PARA['senioridade_lvl']

SENIORIDADE_LVL = {
    "Estagiário": 1,
    "Assistente": 2,
    "Analista Júnior": 3,
    "Analista Pleno": 4,
    "Analista Sênior": 5,
    "Especialista": 6,
    "Líder": 7,
    "Supervisor": 8,
    "Coordenador": 9,
    "Gerente": 10,
    "Head": 11,
    "Diretor": 12,
    "CEO": 13
}

# -----------------------------
# JOINs
# -----------------------------
# Esta parte faz o relacionamento entre as tabelas
# Primeiro, garante que as chaves (id, job_id) sejam do mesmo tipo (string)
df_vagas_filtrado["id"] = df_vagas_filtrado["id"].astype(str)
df_application_filtrado["job_id"] = df_application_filtrado["job_id"].astype(str)
df_prospect_filtrado["job_id"] = df_prospect_filtrado["job_id"].astype(str)

# Faz o primeiro JOIN (esquerdo) entre vagas e application
# Usa a coluna 'id' da vaga e 'job_id' do application
df_join = df_vagas_filtrado.merge(
    df_application_filtrado,
    how="left",               # mantém todas as linhas de vagas, mesmo sem correspondência
    left_on="id",
    right_on="job_id",
    suffixes=("_vaga", "_app")  # adiciona sufixos para evitar nomes de colunas duplicados
)

# Faz o segundo JOIN (também esquerdo) entre o resultado anterior e o prospect
df_join = df_join.merge(
    df_prospect_filtrado,
    how="left",
    left_on="id",
    right_on="job_id",
    suffixes=("", "_prospect")
)

# Remove as colunas de chaves duplicadas geradas pelo JOIN
for col in ["job_id", "job_id_prospect"]:
    if col in df_join.columns:
        df_join = df_join.drop(columns=col)

# -----------------------------
# Coluna job_description_final
# -----------------------------
# Compara as colunas 'principais_atividades' e 'competencia_tecnicas_e_comportamentais'
# Se forem iguais, mantém apenas o valor de 'principais_atividades'
# Se forem diferentes, concatena os dois valores separados por " | "

# Converte para string para evitar erros e normaliza (remove espaços extras)
a = df_join["principais_atividades"].astype("string")
b = df_join["competencia_tecnicas_e_comportamentais"].astype("string")
a_norm = a.fillna("").str.strip()
b_norm = b.fillna("").str.strip()

df_join["job_description_final"] = np.where(
    a_norm == b_norm,
    a,                                   # se iguais, mantém apenas uma
    a.str.cat(b, na_rep="", sep=" | ")   # se diferentes, concatena
)

# -----------------------------
# Colunas de Senioridade
# -----------------------------
# Cria duas novas colunas:
# 1. nivel_profissional_group: usando SENIORIDADE_GROUP
# 2. nivel_profissional_lvl: usando SENIORIDADE_LVL com base no group

df_join["nivel_profissional_group"] = df_join["nivel_profissional"].map(SENIORIDADE_GROUP)
df_join["nivel_profissional_lvl"] = df_join["nivel_profissional_group"].map(SENIORIDADE_LVL)

# -----------------------------
# Inspeção final dos dados
# -----------------------------
# Mostra as 5 primeiras linhas das novas colunas para conferir o resultado
print(df_join[["nivel_profissional", "nivel_profissional_group", "nivel_profissional_lvl"]].head())

# Mostra o tamanho do DataFrame resultante (linhas e colunas)
print(df_join.shape)

# Mostra as 5 primeiras linhas de todo o DataFrame
df_join.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/tmp/ipython-input-324301665.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vagas_filtrado["id"] = df_vagas_filtrado["id"].astype(str)
/tmp/ipython-input-324301665.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_application_filtrado["job_id"] = df_application_filtrado["job_id"].astype(str)
/tmp/ipython-input-324301665.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

  nivel_profissional nivel_profissional_group  nivel_profissional_lvl
0             Sênior          Analista Sênior                       5
1             Sênior          Analista Sênior                       5
2             Sênior          Analista Sênior                       5
3             Sênior          Analista Sênior                       5
4             Sênior          Analista Sênior                       5
(56539, 19)


,id,titulo_vaga,objetivo_vaga,principais_atividades,competencia_tecnicas_e_comportamentais,estado,nivel_academico,nivel_ingles,nivel_espanhol,nivel_profissional,sexo,pcd,cv_pt,prospect_codigo,titulo,prospect_situacao_candidado_normalized,job_description_final,nivel_profissional_group,nivel_profissional_lvl
0,5185,Operation Lead -,None,Operations Lead\n\nRoles & Responsibilities:\n...,Required Skills:\n• Prior experience in Cloud ...,São Paulo,Ensino Superior Completo,Avançado,Fluente,Sênior,None,None,,11010,Operation Lead -,encaminhado ao requisitante,Operations Lead\n\nRoles & Responsibilities:\n...,Analista Sênior,5
1,5184,Consultor PP/QM Sênior,Contratação,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...,• Consultor PP/QM Sênior com experiencia em pr...,São Paulo,Ensino Superior Completo,Fluente,Nenhum,Sênior,None,None,,26770,Consultor PP/QM Sênior,prospect,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...,Analista Sênior,5
2,5184,Consultor PP/QM Sênior,Contratação,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...,• Consultor PP/QM Sênior com experiencia em pr...,São Paulo,Ensino Superior Completo,Fluente,Nenhum,Sênior,None,None,,26759,Consultor PP/QM Sênior,prospect,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...,Analista Sênior,5
3,5184,Consultor PP/QM Sênior,Contratação,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...,• Consultor PP/QM Sênior com experiencia em pr...,São Paulo,Ensino Superior Completo,Fluente,Nenhum,Sênior,None,None,,26758,Consultor PP/QM Sênior,prospect,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...,Analista Sênior,5
4,5184,Consultor PP/QM Sênior,Contratação,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...,• Consultor PP/QM Sênior com experiencia em pr...,São Paulo,Ensino Superior Completo,Fluente,Nenhum,Sênior,None,None,,26757,Consultor PP/QM Sênior,prospect,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...,Analista Sênior,5
